In [2]:
import subprocess
import os
from openai import OpenAI
import re
import ollama

In [3]:

path_to_pvpython = "/Applications/ParaView-5.12.0.app/Contents/bin:$PATH"  # Replace with the actual path returned by `which pvpython`
os.environ["PATH"] += os.pathsep + path_to_pvpython


In [4]:
client = OpenAI(
    api_key="" # provide you OpenAI API key here
)

In [4]:
def extract_python_code(text, name):
    """
    Extracts and prints blocks of Python code from a given text that are delimited by
    ```python and ```
    """
    # Regular expression to find all occurrences of Python code blocks
    code_blocks = re.findall(r"```python(.*?)```", text, re.DOTALL)

    
    for i, block in enumerate(code_blocks, start=1):
        # Strip leading/trailing whitespace and maintain internal formatting
        formatted_block = block.strip()
        # Define file path for each code block
        filename = "/Users/tanwimallick/Documents/Paraview/generated_code/{}_{}.py".format(name, i)

        #filename = f"/Users/tanwimallick/Documents/Paraview/generated_code/" + name + "_{i}.py"
        with open(filename, 'w') as file:
            file.write(formatted_block)
        print(f"Code Block {i} saved to {filename}")
        return filename

In [5]:
def extract_error_messages(stderr_output):
    # Split the stderr output into lines
    lines = stderr_output.split('\n')
    
    # Initialize a list to store error messages
    error_messages = []

    # Extract lines that contain error messages
    for i, line in enumerate(lines):
        if 'Traceback (most recent call last):' in line:
            # Start of a new traceback, find the next line starting with 'File'
            for j in range(i+1, len(lines)):
                if lines[j].strip().startswith('File'):
                    # Add lines until 'AttributeError' or other errors are encountered
                    error_detail = lines[j].strip()
                    k = j + 1
                    while k < len(lines) and not lines[k].strip().startswith('File'):
                        error_detail += '\n' + lines[k].strip()
                        k += 1
                    error_messages.append(error_detail)
                    break

    return error_messages

In [6]:
code_to_read = """
from paraview.simple import *

# read the input data
ml100vtk = LegacyVTKReader(FileNames=<path>)
"""

code_to_slice = """
from paraview.simple import *
# create a new slice
slice1 = Slice(registrationName='Slice1', Input=ml100vtk)
slice1.SliceType = 'Plane'
slice1.HyperTreeGridSlicer = 'Plane'
slice1.SliceOffsetValues = [0.0]
slice1.PointMergeMethod = 'Uniform Binning'
"""

code_to_contour = """
from paraview.simple import *

# create a new contour
contour1 = Contour(registrationName='Contour1', Input=ml100vtk)
contour1.ContourBy = ['POINTS', 'var0']
contour1.Isosurfaces = [0.5]
contour1.PointMergeMethod = 'Uniform Binning'
"""

code_to_clip = """
# create a new clip filter
clip = Clip(registrationName='Clip', Input=delaunay3D)
clip.ClipType = 'Plane'
clip.ClipType.Origin = [0.0, 0.0, 0.0]
clip.ClipType.Normal = [1.0, 0.0, 0.0]
"""

code_to_color_transfer_function = """
# get color transfer function/color map for 'var0'
var0LUT = GetColorTransferFunction('var0')
var0LUT.RGBPoints = [min, 0.0, 0.0, 0.75, (min + max) / 2.0, 0.75, 0.75, 0.75, max, 0.75, 0.0, 0.0]
"""

code_to_opacity_transfer_function = """
# get opacity transfer function/opacity map for 'var0'
var0PWF = GetOpacityTransferFunction('var0')
var0PWF.Points = [min, 0.0, 0.5, 0.0, (min + max) / 2.0, 0.5, 0.5, 0.0, max, 1.0, 0.5, 0.0]
"""

code_to_create_layout = """
# create new layout object
layout = CreateLayout(name='Layout')
layout.AssignView(0, renderView)
"""

code_to_contour1Display = """
# show data
contour1Display = Show(contour1, renderView)
contour1Display.ColorArrayName = ['POINTS', '']
contour1Display.DiffuseColor = [1.0, 0.0, 0.0]
"""

code_to_render_view = """
renderView = CreateView('RenderView')
renderView.ViewSize = [1920, 1080]
"""

code_to_render_view_direction = """
# set render view direction
renderView.ResetActiveCameraToPositiveX()
renderView.ResetCamera()
"""

code_to_isometric_view= """
# set render view direction
renderView.ApplyIsometricView()
renderView.ResetCamera()
"""

code_to_save = """
# Save a screenshot of the render view
SaveScreenshot('/Users/tanwimallick/Documents/Paraview/generated_code/points-surf-clip-screenshot.png',renderView, ImageResolution=[1920, 1080], OverrideColorPalette='WhiteBackground')
"""

In [7]:
exapmle_input = f'''I would like to use ParaView to visualize a dataset.
Please generate a ParaView Python script for the following operations.
Read in the file named '/Users/tanwimallick/Documents/Paraview/generated_code/disk.ex2'.
Trace streamlines of the V data array seeded from a default point cloud.
Render the streamlines with tubes.
Add cone glyphs to the streamlines.
Color the streamlines and glyphs by the Temp data array.
View the result in the +X direction.
Save a screenshot of the result in the filename '/Users/tanwimallick/Documents/Paraview/generated_code/stream-glyph-screenshot.png'.
The rendered view and saved screenshot should be 1920 x 1080 pixels.
'''

exapmle_prompt = f'''
This script uses ParaView to visualize streamlines of the V data array from the disk.ex2 file.
Operations include reading the file, tracing streamlines, rendering with tubes, adding cone glyphs,
coloring by the Temp data array, and viewing from the +X direction.

Requirements:
- Read the file '/Users/tanwimallick/Documents/Paraview/generated_code/disk.ex2'.
- Trace streamlines of the V data array seeded from a default point cloud.
- Render the streamlines with tubes for better visibility.
- Add cone glyphs to the streamlines to indicate direction.
- Color both the streamlines and glyphs using the Temp data array.
- Orient the view to look from the +X direction.
- Save a screenshot of the view at 1920 x 1080 pixels resolution to '/Users/tanwimallick/Documents/Paraview/generated_code/stream-glyph-screenshot.png'.
'''

In [8]:
user_input = f'''I would like to use ParaView to visualize a dataset. 
Please generate a ParaView Python script for the following operations. 
Read in the file named '/Users/tanwimallick/Documents/Paraview/generated_code/ml-100.vtk'. 
Generate a volume rendering using the default transfer function. Rotate the view to an isometric direction. 
Save a screenshot of the result in the filename '/Users/tanwimallick/Documents/Paraview/generated_code/ml-dvr-screenshot.png'. 
The rendered view and saved screenshot should be 1920 x 1080 pixels.'''


In [9]:
chat_completion = client.chat.completions.create(
    messages=[
        {"role": "system", "content": f"You are a prompt generator. Do not provide any other text than the prompt "},
        {"role": "user", "content": f"Generate the most effective prompt for the user input \n{user_input}\n. Here is an  example of \n{exapmle_input}\n and generated prompt \n{exapmle_prompt} \n. "},
    ],
    model="gpt-4o",
)

In [10]:
prompt = chat_completion.choices[0].message.content 
print(prompt)

This script uses ParaView to visualize a dataset using volume rendering from a VTK file.
Operations include reading the file, generating a volume rendering with the default transfer function, rotating the view to an isometric direction, and saving a screenshot.

Requirements:
- Read the file '/Users/tanwimallick/Documents/Paraview/generated_code/ml-100.vtk'.
- Generate a volume rendering using the default transfer function.
- Rotate the view to an isometric direction.
- Ensure that the rendered view and saved screenshot are 1920 x 1080 pixels in resolution.
- Save a screenshot of the volume rendering to '/Users/tanwimallick/Documents/Paraview/generated_code/ml-dvr-screenshot.png'.

Generate a ParaView Python script to fulfill the requirements listed above.


In [11]:
chat_completion = client.chat.completions.create(
    messages=[
        {"role": "system", "content": f"You are a code assistant. Please read the user prompt line-by-line and process it step by step. Some operations are provided as examples: \n{code_to_read}\n {code_to_slice}\n {code_to_contour} \n {code_to_clip}. Use \n{code_to_opacity_transfer_function}\n and {code_to_color_transfer_function}. Use the examples \n{code_to_render_view} \n {code_to_render_view_direction} \n {code_to_isometric_view} and \n{code_to_contour1Display}\n and change the render view as the user is specifying. Please use the example to write the correct code for the user. Please use this code \n{code_to_create_layout}\n in all generated code snippets. Do not use clip1.InsideOut. Save the screenshot using \n{code_to_save}."},
        {"role": "user", "content": prompt},
    ],
    model="gpt-4-turbo" #gpt-4o",
)
script = chat_completion.choices[0].message.content  
file_path = extract_python_code(script, 'ml-dvr')

# The command to run, including the pvpython interpreter and the script path
command = ["pvpython", file_path]

# Execute the command
stderr_text = subprocess.run(command, capture_output=True, text=True).stderr

errors = extract_error_messages(stderr_text)
print(errors)

Code Block 1 saved to /Users/tanwimallick/Documents/Paraview/generated_code/ml-dvr_1.py
[]


In [12]:
# while errors:
#     # Communicate with the AI to get a fix
#     chat_completion = client.chat.completions.create(
#         messages=[
#                 {"role": "system", "content": "You are a code assistant."},
#                 {"role": "user", "content": f"I encountered a Python error:\n{errors}\n Can you fix the code \n{script}\n and write a new script?"},
#         ],
#         model="gpt-4",
#     )
    
#     # Assuming the AI provides new Python code in the response
#     script = chat_completion.choices[0].message.content
#     file_path = extract_python_code(script, 'ml-dvr')
    
#     # Execute the new script with pvpython
#     command = ["pvpython", file_path]
#     result = subprocess.run(command, capture_output=True, text=True)
    
#     # Extract errors from stderr, if any
#     errors = extract_error_messages(result.stderr)
#     if not errors:
#         print("No more errors detected. Script executed successfully.")
#         break
#     else:
#         print("Errors detected. Trying again...")